In [9]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [10]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [11]:
np.random.seed(42)
n_samples = 1000
X = np.random.uniform(-1, 1, (n_samples, 1))
y = np.sin(3 * np.pi * X) + 0.1 * np.random.randn(n_samples, 1)
X_train, y_train = X[:800], y[:800]
X_test, y_test = X[800:], y[800:]

In [12]:
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

In [13]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Linear(1, 100)
        self.layer2 = torch.nn.Linear(100, 1)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.layer2(x)
        return x

In [14]:
def train_and_evaluate(model, epochs, optimizer, criterion):
    train_losses = []
    test_losses = []
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        with torch.no_grad():
            outputs = model(X_test)
            test_loss = criterion(outputs, y_test)
            test_losses.append(test_loss.item())

    return test_losses[-1]

In [ ]:
n_epochs = [i for i in range(10, 2000, 2)]
test_mses = []
for epochs in n_epochs:
    model = Net()
    model = model.to(device)
    optimizer =  torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    
    test_mse = train_and_evaluate(model, epochs, optimizer, criterion)
    test_mses.append(test_mse)
    
plt.plot(n_epochs, test_mses, '-o')
plt.xlabel('No. Samples')
plt.ylabel('Test MSE')
plt.show()